Mushroom Classfication Model: Is the mushroom Poisonous or Ediable?

Preprocessing:

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [44]:
df = pd.read_csv('mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [45]:
X= df.drop(columns=['class','veil-type','veil-color','gill-attachment','stalk-shape'],axis=1)
y= df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-spacing,gill-size,gill-color,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,c,n,k,e,s,s,w,w,o,p,k,s,u
1,x,s,y,t,a,c,b,k,c,s,s,w,w,o,p,n,n,g
2,b,s,w,t,l,c,b,n,c,s,s,w,w,o,p,n,n,m
3,x,y,w,t,p,c,n,n,e,s,s,w,w,o,p,k,s,u
4,x,s,g,f,n,w,b,k,e,s,s,w,w,o,e,n,a,g


Prior Probabilities: probability of p and e in all data

In [46]:
def prior_prob(y_train):
    counts = y_train.value_counts().to_dict()
    total = len(y_train)
    for i in counts:
        counts[i]/=total
    return counts
prior = prior_prob(y_train)
for i in prior:
    print(f'{i}:{prior[i]}')

e:0.5226489414081733
p:0.4773510585918267


In [47]:
def likehoodprob(X_train,y_train):
    class_counts = {
    }
    for i in X_train.columns:
        class_counts[i]={}
        for j in X_train[i].unique():
            class_counts[i][j]={}
            for k in y_train.unique():
                count = sum((X_train[i]==j)& (y_train== k))
                class_counts[i][j][k]=count
    return class_counts
likehood = likehoodprob(X_train,y_train)
def total_likehood(likehood):
    total_count = {}
    for i in likehood:
        total_count[i]={}
        for j in likehood[i]:
            total_count[i][j]=sum(likehood[i][j].values())
    return total_count

total = total_likehood(likehood)
for i in total:
    print(f'{i}:{total[i]}')
for i in likehood:
    print(f'{i}:{likehood[i]}')


cap-shape:{'f': 1567, 'x': 1814, 'k': 447, 'b': 214, 's': 17, 'c': 3}
cap-surface:{'f': 1184, 'y': 1581, 's': 1295, 'g': 2}
cap-color:{'n': 1185, 'p': 73, 'g': 894, 'y': 515, 'w': 524, 'e': 740, 'b': 90, 'c': 29, 'u': 5, 'r': 7}
bruises:{'t': 1684, 'f': 2378}
odor:{'n': 1807, 'l': 186, 'f': 1061, 'y': 290, 'c': 101, 's': 292, 'a': 190, 'p': 113, 'm': 22}
gill-spacing:{'c': 3389, 'w': 673}
gill-size:{'b': 2799, 'n': 1263}
gill-color:{'n': 545, 'w': 619, 'p': 683, 'g': 372, 'b': 883, 'k': 209, 'h': 347, 'u': 253, 'e': 56, 'o': 31, 'r': 13, 'y': 51}
stalk-root:{'b': 1889, '?': 1266, 'c': 272, 'e': 550, 'r': 85}
stalk-surface-above-ring:{'s': 2594, 'k': 1170, 'f': 282, 'y': 16}
stalk-surface-below-ring:{'s': 2503, 'k': 1130, 'y': 135, 'f': 294}
stalk-color-above-ring:{'p': 959, 'e': 50, 'g': 294, 'w': 2222, 'n': 207, 'b': 200, 'o': 102, 'c': 22, 'y': 6}
stalk-color-below-ring:{'g': 292, 'w': 2202, 'p': 930, 'n': 233, 'b': 217, 'o': 102, 'e': 52, 'y': 12, 'c': 22}
ring-number:{'o': 3731, 't

In [48]:
def predict_naive_bayes(X_new, priors, likelihood_table):
    predictions = []
    
    for _, sample in X_new.iterrows():
        posteriors = {}
        
        for cls in priors.keys():
            posterior = priors[cls]
            
            for feature, value in sample.items():
                if value in likelihood_table[feature]:
                    posterior *= likelihood_table[feature][value].get(cls, 1.0)
            
            posteriors[cls] = posterior
        
        predictions.append(max(posteriors.items(), key=lambda x: x[1])[0])
    
    return predictions

predictions = predict_naive_bayes(X_train, prior, likehood)

accuracy = sum(predictions == y_train) / len(y_train)
print(f'Training Accuracy: {accuracy:.2f}')

Training Accuracy: 1.00


In [ ]:

predictions = predict_naive_bayes(X_test, prior, likehood)

accuracy = sum(predictions == y_test) / len(y_test)
print(f'Testing Accuracy: {accuracy:.2f}')

Testing Accuracy: 1.00


In [52]:
import joblib

# Assuming you have a dictionary with class probabilities and feature probabilities
model_data = {
    "class_probs": prior,  # Prior probabilities
    "feature_probs": likehood,  # Conditional probabilities
    "predict_function": predict_naive_bayes # Your custom function
}

# Save everything to a file
joblib.dump(model_data, "mushroom_classifier.pkl")
print("Model saved successfully!")


Model saved successfully!
